In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.models import Model, load_model
from keras.layers import Input, Dense, BatchNormalization, Activation, AveragePooling2D, Flatten
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, LearningRateScheduler
from keras.datasets import cifar100
from keras import backend as K
import pickle
import sklearn
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances
%matplotlib inline
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [2]:
model_type = 'ResNet56v2'
embedding_type = 'node2vec'
seed = 1

In [3]:
def load_embeddings(fname):   
    file_path = os.path.join(os.getcwd(), 'wiki/%s.emb' % (fname))
    
    embeddings = {}
    with open(file_path) as file:
        for line in file:
            line = line.strip().split()
            if int(line[0]) >= 100:
                continue
            embeddings[int(line[0])] = np.array(line[1:]).astype(float)

    return embeddings

label_embeddings = load_embeddings('link_edges')

In [4]:
cat_dict = pickle.load(open(os.path.join(os.getcwd(), 'wiki/link_edges_dict_cat.pkl'), "rb"))
graph_to_cifar_dict = pickle.load(open(os.path.join(os.getcwd(), 'wiki/graph_to_cifar_labels.pkl'), "rb"))

data_dir = os.path.join(os.getcwd(), 'Data/cifar-100-python/')
meta = pickle.load(open(os.path.join(data_dir, 'meta'), "rb"), encoding='latin1')
fine_label_names = meta['fine_label_names']

In [5]:
#Loading data and embeddings
embedding_len = len(label_embeddings[0])

label_embeddings_arr = np.zeros((100, embedding_len))
for i in range(100):
    label_embeddings_arr[i] = label_embeddings[i]

save_dir_feat = os.path.join(os.getcwd(), 'saved_models/zsl/%s/type1/seed%s/extracted_feat/' % (model_type, seed))

In [6]:
X_train_feat = np.load(os.path.join(save_dir_feat, 'X_train_feat_cifar100_%s.npy' % (model_type)))
X_test_seen_feat = np.load(os.path.join(save_dir_feat, 'X_test_seen_feat_cifar100_%s.npy' % (model_type)))
X_test_unseen_feat = np.load(os.path.join(save_dir_feat, 'X_test_unseen_feat_cifar100_%s.npy' % (model_type)))
X_test_all_feat = np.load(os.path.join(save_dir_feat, 'X_test_all_feat_cifar100_%s.npy' % (model_type)))

input_shape = X_train_feat[0].shape

In [7]:
y_train = np.load(os.path.join(save_dir_feat, 'y_train_cifar100_%s.npy' % (model_type)))
y_test_seen = np.load(os.path.join(save_dir_feat, 'y_test_seen_cifar100_%s.npy' % (model_type)))
y_test_unseen = np.load(os.path.join(save_dir_feat, 'y_test_unseen_cifar100_%s.npy' % (model_type)))
y_test_all = np.load(os.path.join(save_dir_feat, 'y_test_all_cifar100_%s.npy' % (model_type)))

In [8]:
y_train_embeddings = np.zeros((len(y_train), embedding_len))
for i in range(len(y_train)):
    y_train_embeddings[i] = label_embeddings[int(y_train[i])]
    
y_test_seen_embeddings = np.zeros((len(y_test_seen), embedding_len))
for i in range(len(y_test_seen)):
    y_test_seen_embeddings[i] = label_embeddings[int(y_test_seen[i])]
    
y_test_unseen_embeddings = np.zeros((len(y_test_unseen), embedding_len))
for i in range(len(y_test_unseen)):
    y_test_unseen_embeddings[i] = label_embeddings[int(y_test_unseen[i])]
    
y_test_all_embeddings = np.zeros((len(y_test_all), embedding_len))
for i in range(len(y_test_all)):
    y_test_all_embeddings[i] = label_embeddings[int(y_test_all[i])]

In [9]:
def build_model(input_shape=input_shape, embedding_len=embedding_len):
    inputs = Input(shape=input_shape)

    x = BatchNormalization()(inputs)
    x = Activation('relu')(x)
    x = AveragePooling2D(pool_size=2)(x)
    
    x = Flatten()(x)
    
    x = Dense(1024) (x)
    x = Activation('relu')(x)
    x = BatchNormalization()(x)
    
    x = Dense(512) (x)
    x = Activation('relu')(x)
    x = BatchNormalization()(x)
    
    x = Dense(embedding_len) (x)
    x = Activation('relu')(x)
    x = BatchNormalization()(x)
    
    outputs = Dense(embedding_len,
                    kernel_initializer='he_normal')(x)
    
    model = Model(inputs=inputs, outputs=outputs)
    
    return model

In [10]:
batch_size = 128
epochs = 40

In [11]:
model = build_model()
model.compile(loss='cosine_proximity', optimizer='adam', metrics=['mse'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 8, 8, 256)         0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 8, 8, 256)         1024      
_________________________________________________________________
activation_1 (Activation)    (None, 8, 8, 256)         0         
_________________________________________________________________
average_pooling2d_1 (Average (None, 4, 4, 256)         0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 4096)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              4195328   
_________________________________________________________________
activation_2 (Activation)    (None, 1024)              0         
__________

In [12]:
save_dir = os.path.join(os.getcwd(), 'saved_models/zsl/%s/type1/seed%s/%s_transfer/' % (model_type, seed, embedding_type))
model_name = 'cifar100_%s_model.{epoch:03d}.h5' % (model_type)
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
filepath = os.path.join(save_dir, model_name)

In [13]:
def lr_schedule(epoch):
    """Learning Rate Schedule
    Learning rate is scheduled to be reduced after 80, 120, 160, 180 epochs.
    Called automatically every epoch as part of callbacks during training.
    # Arguments
        epoch (int): The number of epochs
    # Returns
        lr (float32): learning rate
    """
    lr = 1e-3
    if epoch > 120:
        lr *= 0.5e-3
    elif epoch > 80:
        lr *= 1e-3
    elif epoch > 40:
        lr *= 1e-2
    elif epoch > 20:
        lr *= 1e-1
    print('Learning rate: ', lr)

    return lr

In [14]:
checkpoint = ModelCheckpoint(filepath=filepath,
                             monitor='val_loss',
                             verbose=1,
                             save_best_only=True)

lr_scheduler = LearningRateScheduler(lr_schedule)

lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                               cooldown=0,
                               patience=5,
                               min_lr=0.5e-6)

callbacks = [checkpoint, lr_reducer, lr_scheduler]

In [15]:
model.fit(X_train_feat, y_train_embeddings,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(X_test_seen_feat, y_test_seen_embeddings),
          shuffle=True,
          callbacks=callbacks,
          verbose=1
         )

Train on 40000 samples, validate on 8000 samples
Epoch 1/40
Learning rate:  0.001
40000/40000 [==============================] - 6s 151us/step - loss: -0.7800 - mean_squared_error: 2.3531 - val_loss: -0.8724 - val_mean_squared_error: 1.4993

Epoch 00001: val_loss improved from inf to -0.87242, saving model to /home/tliu/Dev/CMU/10-715/10715_Project/saved_models/zsl/ResNet56v2/type1/seed1/node2vec_transfer/cifar100_ResNet56v2_model.001.h5
Epoch 2/40
Learning rate:  0.001
40000/40000 [==============================] - 5s 122us/step - loss: -0.8935 - mean_squared_error: 0.6216 - val_loss: -0.9008 - val_mean_squared_error: 0.3915

Epoch 00002: val_loss improved from -0.87242 to -0.90084, saving model to /home/tliu/Dev/CMU/10-715/10715_Project/saved_models/zsl/ResNet56v2/type1/seed1/node2vec_transfer/cifar100_ResNet56v2_model.002.h5
Epoch 3/40
Learning rate:  0.001
40000/40000 [==============================] - 5s 122us/step - loss: -0.9076 - mean_squared_error: 0.4212 - val_loss: -0.9072 -

KeyboardInterrupt: 

In [16]:
df1 = pd.DataFrame([cat_dict.keys(), cat_dict.values()]).transpose()
df1.columns = ['graph_label', 'graph_label_name']

df2 = pd.DataFrame([graph_to_cifar_dict.keys(), graph_to_cifar_dict.values()]).transpose()
df2.columns = ['graph_label_name', 'cifar_label_name']

df3 = pd.DataFrame([fine_label_names]).transpose().reset_index()
df3.columns = ['cifar_label', 'cifar_label_name']

df_labels = df1.merge(df2, left_on='graph_label_name', right_on='graph_label_name')
df_labels = df_labels.merge(df3, left_on='cifar_label_name', right_on='cifar_label_name')

label_dict = df_labels[['graph_label', 'cifar_label']].set_index('graph_label').to_dict()['cifar_label']

In [17]:
#Top K Predictions
def pred_top_k(y_test_pred, label_embeddings_arr=label_embeddings_arr, k=5):
    sim_table = cosine_similarity(y_test_pred, label_embeddings_arr)
    top_k_guesses = np.argpartition(sim_table,range(99-k+1,100),axis=1)[:,99-k+1:]
    return sim_table, top_k_guesses

#Top k Accuracy
def calc_top_k_acc(top_k, y_test):
    correct = 0
    for i in range(y_test.shape[0]):
        if np.squeeze(y_test)[i] in top_k[i]:
            correct += 1
    return correct/float(y_test.shape[0])

def evaluate(model, x_test, y_test, label_embeddings_arr, k=5):
    y_test_pred = model.predict(x_test)
    sim_table, top_k_guesses = pred_top_k(y_test_pred, label_embeddings_arr, k=k)
    top_k_guesses = [[label_dict[i] for i in top_k_guesses[j]] for j in range(len(top_k_guesses))]
    
    #Top prediction
    label_predictions = sim_table.argmax(axis=1)
    label_predictions = [label_dict[i] for i in label_predictions]
    
    #Accuracy
    acc = np.sum((np.squeeze(y_test) == label_predictions)) / float(y_test.shape[0])
    top_k_acc = calc_top_k_acc(top_k_guesses, y_test)
    print("Accuracy: " + str(acc))
    print("Top " + str(k) + " Accuracy: " + str(top_k_acc))

In [18]:
best_model_filepath = os.path.join(save_dir, 'cifar100_%s_model.%03d.h5' % (model_type, 19))
best_model = load_model(best_model_filepath)

In [19]:
evaluate(best_model, X_test_seen_feat, y_test_seen, label_embeddings_arr)

Accuracy: 0.077
Top 5 Accuracy: 0.12325


In [20]:
# Although accuracy is still 0%, we seen an improvement in top 5 % accuracy (0% -> 15%). Sign that w2v is useful for ZSL

evaluate(best_model, X_test_unseen_feat, y_test_unseen, label_embeddings_arr)

Accuracy: 0.01
Top 5 Accuracy: 0.0705


In [21]:
# Regular ZSL setting where we only need to consider choose between the 20 unseen classes, rather than all 100

# Just replace the word vectors for "seen" labels with something really far so it won't be close to any predicted vector
unseen_labels = np.unique(y_test_unseen)
label_embeddings_arr_unseen = np.copy(label_embeddings_arr)
for i in range(100):
    if label_dict[i] in unseen_labels:
        continue
    label_embeddings_arr_unseen[i] = np.ones(label_embeddings_arr[0].shape) * 1000

evaluate(best_model, X_test_unseen_feat, y_test_unseen, label_embeddings_arr_unseen) # Significant improvement

Accuracy: 0.05458333333333333
Top 5 Accuracy: 0.26166666666666666


In [22]:
"""
x = np.linspace(0,99,100) 
y = np.zeros(100)
y_test_pred = model.predict(x_test)

for i in range(100):
    k = i+1
    sim_table, top_k_guesses = pred_top_k(y_test_pred, k=k)
    y[i] = calc_top_k_acc(top_k_guesses, y_test)
    
plt.plot(x,y)
plt.title("Top K Accuracy")
"""

'\nx = np.linspace(0,99,100) \ny = np.zeros(100)\ny_test_pred = model.predict(x_test)\n\nfor i in range(100):\n    k = i+1\n    sim_table, top_k_guesses = pred_top_k(y_test_pred, k=k)\n    y[i] = calc_top_k_acc(top_k_guesses, y_test)\n    \nplt.plot(x,y)\nplt.title("Top K Accuracy")\n'